In [ ]:
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine

aujourdhui = datetime.now()
date_controle = aujourdhui.strftime('%d-%m-%y')


user = 'airflow'
password = 'airflow'
host = 'postgres'
port = '5432' 
database_bronze = 'bronze'
database_silver = 'silver'

db_url_bronze = f"postgresql://{user}:{password}@{host}:{port}/{database_bronze}"
engine_bronze = create_engine(db_url_bronze)

db_url_silver = f"postgresql://{user}:{password}@{host}:{port}/{database_silver}"
engine_silver= create_engine(db_url_silver)


df_clients = pd.read_sql("SELECT * FROM clients", con=engine_bronze)
df_controles = pd.read_sql("SELECT * FROM controles", con=engine_silver)


In [2]:
df_clients.head()

,ID_CLIENT,NOM,PRENOM,CIN,CARTE_SEJOUR,VILLE,NATIONALITE,TELEPHONE,id_intermediaire,DATE_CREATION
0,lu1cldwMLx,Hicks,Dawn,SJ109379,OL118027,None,mre,800-778-8075x457,84,2021-05-14
1,k8N4sn4YJb,Wolfe,Kimberly,B136474,O499923,North Alishaport,mre,+1-913-505-2118x425,15,2024-03-27
2,6qzPbzht3H,Burch,Isabel3@,KX010301,HM264290,North Eric`,mre,935-921-5809,13,2021-07-16
3,KZPbHdxc1a,None,Matthew,YT977001,OI901480,South Kenneth,etranger,4332103507,2,2023-02-25
4,3KaFE6Zhq9,Roth=#,None,EB353024,B240907,None,marocain,001-367-758-8791,91,2021-01-03


In [3]:
df_controles.head()

,ID_CONTROLE,DATA,DIM_CONTROLE,DATE_CONTROLE


### table resultat unicite de la colonne cin , nom, et prenom

In [ ]:
def tester_unicite_cin_nom_prenom(row):
    cin = row['cin']
    nom = row['nom']
    prenom = row['prenom']

    count = df_clients[
        (df_clients['cin'] == cin) & 
        ((df_clients['nom'] != nom) | (df_clients['prenom'] != prenom))
    ].shape[0]
    
    if count > 0:
        return 0  
    return 1  


In [ ]:
df_UN_cin_nom_prenom = df_clients[['id_client']].copy()
df_UN_cin_nom_prenom['result'] = df_clients.apply(tester_unicite_cin_nom_prenom, axis=1)

max_id = df_controles['id_controle'].max()
df_UN_cin_nom_prenom['id_controle'] = max_id + 1

In [ ]:
df_UN_cin_nom_prenom.head()

In [ ]:
df_UN_cin_nom_prenom.to_sql('df_un_cin_nom_prenom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "cin_nom_prenom"
dim_controle = "unicite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat unicite de la colonne carte sejour , nom, et prenom

In [ ]:
def tester_unicite_carte_nom_prenom(row):
    carte_sejour = row['carte_sejour']
    nom = row['nom']
    prenom = row['prenom']
    
 
    count = df_clients[
        (df_clients['carte_sejour'] == carte_sejour) & 
        ((df_clients['nom'] != nom) | (df_clients['prenom'] != prenom))
    ].shape[0]
    
    if count > 0:
        return 0  
    return 1  


In [ ]:
df_UN_carte_nom_prenom = df_clients[['id_client']].copy()
df_UN_carte_nom_prenom['result'] = df_clients.apply(tester_unicite_carte_nom_prenom, axis=1)


max_id = df_controles['id_controle'].max()
df_UN_carte_nom_prenom['id_controle'] = max_id + 1


In [ ]:
df_UN_carte_nom_prenom.to_sql('df_un_carte_nom_prenom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "carte_sejour_nom_prenom"
dim_controle = "unicite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

### table resultat unicite de la colonne telephone , nom, et prenom

In [ ]:
def tester_unicite_tele_nom_prenom(row):
    telephone = row['telephone']
    nom = row['nom']
    prenom = row['prenom']

    count = df_clients[
        (df_clients['telephone'] == telephone) & 
        ((df_clients['nom'] != nom) | (df_clients['prenom'] != prenom))
    ].shape[0]
    
    if count > 0:
        return 0
    return 1  


In [ ]:
df_UN_tele_nom_prenom = df_clients[['id_client']].copy()
df_UN_tele_nom_prenom['result'] = df_clients.apply(tester_unicite_tele_nom_prenom, axis=1)


max_id = df_controles['id_controle'].max()
df_UN_tele_nom_prenom['id_controle'] = max_id + 1


In [ ]:
df_UN_tele_nom_prenom.to_sql('df_un_tele_nom_prenom', con=engine_silver, if_exists='replace', index=False)

### remplissage de la table des controles

In [ ]:
id_controle = df_controles['id_controle'].max() + 1
data = "telephone_nom_prenom"
dim_controle = "unicite"

nouvelles_lignes = [[id_controle, data, dim_controle, date_controle]]
nouvelle_ligne_df = pd.DataFrame(nouvelles_lignes, columns=['id_controle', 'data', 'dim_controle', 'date_controle'])
df_controles = pd.concat([df_controles, nouvelle_ligne_df], ignore_index=True)

In [ ]:
df_controles.to_sql('controles', con=engine_silver, if_exists='append', index=False)